# 邏輯迴歸

在這個不計分的實驗中，你將：
- 探索 Sigmoid 函數（又稱邏輯函數）
- 探索邏輯迴歸（Logistic Regression），它會使用 Sigmoid 函數

In [ ]:
import sys, os
import numpy as np
try:
    %matplotlib widget
except:
    %matplotlib inline
    print("Colab not support matplotlib widget")
import matplotlib.pyplot as plt
from pathlib import Path

#region 匯入資料
def find_repo_root(marker="README.md"):
    cur = Path.cwd()
    while cur != cur.parent:  # 防止無限迴圈，到達檔案系統根目錄就停
        if (cur / marker).exists():
            return cur
        cur = cur.parent
    return None


def import_data_from_github():
    import os, urllib.request, pathlib, shutil
    
    def isRunningInColab() -> bool:
        return "google.colab" in sys.modules

    def isRunningInJupyterLab() -> bool:
        try:
            import jupyterlab
            return True
        except ImportError:
            return False
        
    def detect_env():
        from IPython import get_ipython
        if isRunningInColab():
            return "Colab"
        elif isRunningInJupyterLab():
            return "JupyterLab"
        elif "notebook" in str(type(get_ipython())).lower():
            return "Jupyter Notebook"
        else:
            return "Unknown"
        
    def get_utils_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/utils"
        else:
            return Path.cwd() / "utils"

    env = detect_env()
    UTILS_DIR = get_utils_dir(env)
    REPO_DIR = "Machine-Learning-Lab"

    #shutil.rmtree(UTILS_DIR, ignore_errors=True)
    os.makedirs(UTILS_DIR, exist_ok=True)

    BASE = f"https://raw.githubusercontent.com/mz038197/{REPO_DIR}/main"
    urllib.request.urlretrieve(f"{BASE}/utils/lab_utils_common_classification.py", f"{UTILS_DIR}/lab_utils_common_classification.py")
    urllib.request.urlretrieve(f"{BASE}/utils/plt_one_addpt_onclick.py", f"{UTILS_DIR}/plt_one_addpt_onclick.py")
    urllib.request.urlretrieve(f"{BASE}/utils/deeplearning.mplstyle", f"{UTILS_DIR}/deeplearning.mplstyle")


repo_root = find_repo_root()

if repo_root is None:
    import_data_from_github()
    repo_root = Path.cwd()
    

os.chdir(repo_root)
print(f"✅ 切換工作目錄至 {Path.cwd()}")
sys.path.append(str(repo_root)) if str(repo_root) not in sys.path else None
print(f"✅ 加入到系統路徑")

from utils.lab_utils_common_classification import draw_vthresh
from utils.plt_one_addpt_onclick import plt_one_addpt_onclick

plt.style.use('utils/deeplearning.mplstyle')
print("✅ 匯入模組及設定繪圖樣式")
#endregion

<br>

## Sigmoid（S 形）／邏輯函數

如同課程影片中所討論的，對於分類任務，我們可以先從線性迴歸模型開始：

$f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b$，用來在給定 \(x\) 的情況下預測 \(y\)。

- 但在分類問題中，我們希望模型輸出的預測值落在 0 到 1 之間，因為輸出變數 \(y\) 只有 0 或 1。

- 這可以透過「Sigmoid 函數」達成：它會把所有輸入值映射到 0 到 1 之間。

接著我們來實作 Sigmoid 函數，並親自觀察它的行為。

## Sigmoid 函數的公式

Sigmoid 函數的公式如下：  

$$
\begin{equation}
g(z) = \frac{1}{1+e^{-z}}\tag{1}
\end{equation}
$$ 


在邏輯迴歸中，\(z\)（Sigmoid 的輸入）是線性模型的輸出。

- 若只有一筆資料，\(z\) 是純量（scalar）。

- 若有多筆資料，\(z\) 可能是包含 \(m\) 個值的向量（vector），每個值對應一筆資料。

- 因此 Sigmoid 的實作需要同時支援純量與向量這兩種輸入形式。

我們用 Python 來實作看看。

NumPy 提供了 [`exp()`](https://numpy.org/doc/stable/reference/generated/numpy.exp.html) 函數，可以很方便地對輸入陣列 `z` 的每個元素計算指數（\(e^{z}\)）。

它也支援單一數值作為輸入，如下所示。

In [ ]:
# Input is an array. 
input_array = np.array([1,2,3])
exp_array = np.exp(input_array)

print("Input to exp:", input_array)
print("Output of exp:", exp_array)

# Input is a single number
input_val = 1  
exp_val = np.exp(input_val)

print("Input to exp:", input_val)
print("Output of exp:", exp_val)

`sigmoid` 函數在 Python 中的實作如下（見下一個程式碼 cell）。

In [ ]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Args:
        z (ndarray): A scalar, numpy array of any size.

    Returns:
        g (ndarray): sigmoid(z), with the same shape as z
         
    """

    g = 1/(1+np.exp(-z))
   
    return g

我們來看看當 `z` 取不同數值時，這個函數的輸出會是什麼。

In [ ]:
# Generate an array of evenly spaced values between -10 and 10
z_tmp = np.arange(-10,11)

# Use the function implemented above to get the sigmoid values
y = sigmoid(z_tmp)

# Code for pretty printing the two arrays next to each other
np.set_printoptions(precision=3) 
print("Input (z), Output (sigmoid(z))")
print(np.c_[z_tmp, y])

左欄是 `z`，右欄是 `sigmoid(z)`。可以看到，Sigmoid 的輸入從 -10 到 10，而輸出會落在 0 到 1 之間。

接著我們用 `matplotlib` 把這個函數畫出來。

In [ ]:
# Plot z vs sigmoid(z)
fig,ax = plt.subplots(1,1,figsize=(5,3))
ax.plot(z_tmp, y, c="b")

ax.set_title("Sigmoid function")
ax.set_ylabel('sigmoid(z)')
ax.set_xlabel('z')
draw_vthresh(ax,0)

可以看到，當 `z` 趨近很大的負值時，Sigmoid 會趨近 `0`；當 `z` 趨近很大的正值時，Sigmoid 會趨近 `1`。


<br>

## 邏輯迴歸模型

邏輯迴歸模型會把 Sigmoid 套用到熟悉的線性迴歸模型上，如下所示：

$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = g(\mathbf{w} \cdot \mathbf{x}^{(i)} + b ) \tag{2} $$ 

其中

$$
\begin{equation}
g(z) = \frac{1}{1+e^{-z}}\tag{3}
\end{equation}
$$ 



  
我們把邏輯迴歸套用到「腫瘤分類」這個類別資料的範例。

首先，載入訓練資料，以及參數的初始值。
  


In [ ]:
x_train = np.array([0., 1, 2, 3, 4, 5])
y_train = np.array([0,  0, 0, 1, 1, 1])

w_in = np.zeros((1))
b_in = 0

請嘗試以下步驟：

- 點擊「Run Logistic Regression」，為目前的訓練資料找出最佳的邏輯迴歸模型

    - 注意：得到的模型能相當好地配適資料。

    - 注意：橘色線是上方的 `$z$`，也就是 $\mathbf{w} \cdot \mathbf{x}^{(i)} + b$。它不會和線性迴歸模型中的直線相同。

接著可以透過套用 *threshold*（閾值）來進一步改善結果。

- 勾選「Toggle 0.5 threshold」，看看套用閾值後的預測結果。

    - 這些預測看起來不錯，與資料相當吻合。

    - 接著在腫瘤尺寸較大的範圍（接近 10）再新增一些資料點，並重新執行邏輯迴歸。

    - 和線性迴歸模型不同，這個模型仍然能維持正確的預測

In [ ]:
plt.close('all') 
addpt = plt_one_addpt_onclick( x_train,y_train, w_in, b_in, logistic=True)

## 恭喜！
你已經探索了 Sigmoid 函數在邏輯迴歸中的使用方式。